In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf

def load_agricultural_data(image_dir, mask_dir, target_size=(256, 256)):
    """
    Load images and segmentation masks for agricultural segmentation.
    
    Args:
        image_dir (str): Directory containing aerial images.
        mask_dir (str): Directory containing corresponding segmentation masks.
        target_size (tuple): Desired size of the images and masks (default: 256x256).
        
    Returns:
        numpy array: Preprocessed images and masks.
    """
    images, masks = [], []
    for file_name in os.listdir(image_dir):
        # Image and mask paths
        image_path = os.path.join(image_dir, file_name)
        mask_path = os.path.join(mask_dir, file_name.replace('.jpg', '.png'))  # Match extensions

        # Load and resize image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Unable to load image {image_path}")
            continue
        image = cv2.resize(image, target_size) / 255.0  # Normalize image

        # Load and resize mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale
        if mask is None:
            print(f"Warning: Unable to load mask {mask_path}")
            continue
        mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)

        # Debug: Check unique mask values
        print(f"Unique mask values before correction: {np.unique(mask)}")

        # Handle out-of-range values
        mask[mask >= 4] = 0  # Replace invalid labels with background

        # Convert to one-hot encoding
        mask = tf.keras.utils.to_categorical(mask, num_classes=4)

        images.append(image)
        masks.append(mask)

    return np.array(images), np.array(masks)

# Set dataset paths
image_dir = "Plantations_Segmentation/img"
mask_dir = "Class_Segmentation"

# Load training and validation data
train_images, train_masks = load_agricultural_data(image_dir, mask_dir)


Unique mask values before correction: [ 0 90]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [ 0 90]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [ 0 90]


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

def build_simple_unet(input_shape, num_classes):
    """
    Build a simplified U-Net model for semantic segmentation.

    Args:
        input_shape (tuple): Shape of input images, e.g., (256, 256, 3).
        num_classes (int): Number of output classes for segmentation.

    Returns:
        tensorflow.keras.Model: Simplified U-Net model.
    """
    inputs = Input(input_shape)

    # Downsampling
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)

    # Upsampling
    u4 = UpSampling2D((2, 2))(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)

    u5 = UpSampling2D((2, 2))(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)

    # Output layer
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c5)

    return Model(inputs, outputs)

# Build and compile the model
input_shape = (256, 256, 3)
num_classes = 4  # Replace with the number of your classes
model = build_simple_unet(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load data using the previously defined function
image_dir = "Plantations_Segmentation/img"
mask_dir = "Class_Segmentation"
train_images, train_masks = load_agricultural_data(image_dir, mask_dir)

# Placeholder validation split
split_idx = int(0.8 * len(train_images))
val_images, val_masks = train_images[split_idx:], train_masks[split_idx:]
train_images, train_masks = train_images[:split_idx], train_masks[:split_idx]

# Train the model
history = model.fit(
    train_images, train_masks,
    validation_data=(val_images, val_masks),
    batch_size=2,
    epochs=7
)

# Save the model
model.save('simple_drone_segmentation_model.h5')


Unique mask values before correction: [ 0 90]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [ 0 90]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [  0 170]
Unique mask values before correction: [ 0 90]
Epoch 1/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 356ms/step - accuracy: 0.5247 - loss: 1.3104 - val_accuracy: 1.0000 - val_loss: 0.3986
Epoch 2/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 1.0000 - loss: 0.2342 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 3/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 1.0000 - loss: 5.5064e-04 - val_accuracy: 1.0000 - val_loss: 1.8452e-05
Epoch 4/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 298ms/step - accuracy: 1.0000 - loss: 2.5469e-05 - val_ac